In [1]:
from jinja2 import Environment, PackageLoader, select_autoescape, meta
import pandas as pd, boto3, re, pytz, numpy as np
from datetime import datetime
from tabulate import tabulate
from sparknlp.base import *
from sparknlp.pretrained import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
pd.set_option("display.max_rows",1000)
pd.set_option('display.max_columns', None)
import datetime

In [48]:
#all_models_meta = pd.read_csv("docs_module/metadata/models_metadata_all.csv")
all_classes_meta = pd.read_csv("docs_module/metadata/class_metadata_all.csv")
#full_meta = pd.merge(all_models_meta[all_models_meta.include==1], all_classes_meta, on="Model Class", how=("left"), suffixes=("","class"))

In [62]:
all_classes_meta

,type,approach_class,Model Class,class_description,Input Labels,Output Labels,class_license,dataset_schema,class_annotation_sample,tags,class_parameters,reference_url,paper_url,scala_docs,scala_source,scala_source_tests,evaluator_class,paper_abstract,scala_nlp_code,python_nlp_code,python_nlu_code,wikipedia_link
0,assertion_dl,AssertionDLApproach,AssertionDLModel,Assertion of Clinical Entities based on Deep L...,"document, chunk, word_embeddings",assertion,licensed,NaN,NaN,"clinical,assertion,dl",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,assertion_logreg,AssertionLogRegApproach,AssertionLogRegModel,Assertion of Clinical Entities based on Logist...,"document, chunk, word_embeddings",assertion,licensed,NaN,NaN,"clinical,assertion,ml,logreg",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chunk_entity_resolver,ChunkEntityResolverApproach,ChunkEntityResolverModel,Entity Resolution model Based on KNN using Wor...,"token, chunk_embeddings",entity,licensed,NaN,NaN,"clinical,entity,resolution",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,deidentification,DeIdentification,DeIdentificationModel,Anonymization and DeIdentification model based...,"document, token, chunk",document,licensed,NaN,NaN,"clinical,deidentification,rule based",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,relation_extraction,RelationExtractionApproach,RelationExtractionModel,Relation Extraction model based on syntactic f...,"word_embeddings, chunk, pos, dependency",category,licensed,NaN,NaN,"clinical,relation,extraction",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,default_chunker,NaN,Chunker,This annotator matches a pattern of part-of-sp...,"document, pos",chunk,open source,NaN,NaN,"ner,named entity recognition, chunking","[{'param_name': 'inputCols', 'param_descriptio...",NaN,NaN,https://nlp.johnsnowlabs.com/api/index#com.joh...,https://github.com/JohnSnowLabs/spark-nlp/tree...,https://github.com/JohnSnowLabs/spark-nlp/blob...,NaN,NaN,NaN,NaN,NaN,NaN
6,ngram,NaN,NGramGenerator,Integrates Spark ML NGram function into Spark ...,"token, pos",ngrams,open source,NaN,NaN,"ngram, n-gramm, chunking, shingel,","[{'param_name': 'n', 'param_description': 'num...",NaN,NaN,https://nlp.johnsnowlabs.com/api/index#com.joh...,https://github.com/JohnSnowLabs/spark-nlp/blob...,https://github.com/JohnSnowLabs/spark-nlp/blob...,NaN,NaN,NaN,NaN,NaN,https://en.wikipedia.org/wiki/N-gram
7,classifier_dl,ClassifierDLApproach,ClassifierDLModel,Multi-class Text Classification. ClassifierDL ...,"sentence_embeddings, label",category,open source,NaN,NaN,"classification, deep learning","[{'param_name': 'classes', 'param_description'...",NaN,NaN,https://nlp.johnsnowlabs.com/api/index#com.joh...,https://github.com/JohnSnowLabs/spark-nlp/blob...,https://github.com/JohnSnowLabs/spark-nlp/blob...,NaN,NaN,NaN,NaN,NaN,NaN
8,multi_classifier,MultiClassifierDLApproach,MultiClassifierDLModel,Multi-label Text Classification. MultiClassifi...,"sentence_embeddings, label",category,open source,NaN,NaN,"classification,multi class classification","[{'param_name': 'threshold', 'param_descriptio...",NaN,NaN,https://nlp.johnsnowlabs.com/api/index#com.joh...,https://github.com/JohnSnowLabs/spark-nlp/blob...,https://github.com/JohnSnowLabs/spark-nlp/blob...,NaN,NaN,NaN,NaN,NaN,NaN
9,sentiment_dl,SentimentDLApproach,SentimentDLModel,Multi-class Sentiment Analysis Annotator. Sent...,"sentence, label, sentence_embeddings",category,open source,NaN,NaN,"sentiment classification, sentiment, classific...","[{'param_name': 'threshold', 'param_descriptio...",NaN,NaN,https://nlp.johnsnowlabs.com/api/index#com.joh...,https://github.com/JohnSnowLabs/spark-nlp/blob...,https://github.com/JohnSnowLabs/spark-nlp/blob...,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
full_meta = pd.read_csv("docs_module/metadata/all_models_metadata.csv")
print (full_meta.shape)
#full_meta['input_labels'] = ''
#full_meta = full_meta[(full_meta['include']==1) & (full_meta['model_class']=='NerDLModel')]
#full_meta['automated_output_generation'] = full_meta['Automated_Output_generation']
full_meta = full_meta[(full_meta['include']==1)]
full_meta = full_meta.rename(columns={'model_class':'Model Class'})

full_meta = pd.merge(full_meta, all_classes_meta, on="Model Class", how=("left"), suffixes=("","class"))




(466, 29)


In [56]:
def func(x):
    if 'Ner' in x:
        return 'ner'
    elif 'Perceptron' in x:
        return 'Part of Speech tagging'
    elif 'Lemmatizer'.lower() in x.lower():
        return 'lemmatizer'
    elif 'Embeddings'.lower() in x.lower():
        return 'embeddings'
    elif 'StopWordsCleaner'.lower() in x.lower():
        return 'Stop Words Cleaner'
    elif 'sentiment' in x.lower():
        return 'sentiment'
    elif 'Classifier'.lower() in x.lower():
        return 'Classify <<tweets etc>>'
    elif 'assertion'.lower() in x.lower():
        return 'assertion'
    elif 'EntityResolver'.lower() in x.lower():
        return 'entity_resolver'
    elif 'RelationExtraction'.lower() in x.lower():
        return 'relation_extraction'
    elif 'ContextSpellCheckerModel'.lower() in x.lower():
        return 'spell_checker'
    elif 'PipelineModel'.lower() in x.lower():
        return 'pipeline'
    elif 'DeIdentification'.lower() in x.lower():
        return 'deidentification'
    elif "SentenceDetector".lower() in x.lower():
        return 'sentence_detector'
    return ''
#df['tags'] = df['model_class'].apply(func)

In [57]:
env = Environment(
    loader=PackageLoader('docs_module', 'templates'),
    autoescape=select_autoescape(['html', 'xml'])
)

In [58]:
mdsrc = env.loader.get_source(env, 'model.md')[0]
parsed_content = env.parse(mdsrc)
#meta.find_undeclared_variables(parsed_content)

In [59]:
mdmd = env.get_template("model.md")

In [60]:
full_meta.fillna("",inplace=True)

In [75]:
def generate_code(x, scala=False):
    ins = ",".join([f'"{y.strip()}"' for y in x['Input Labels'].split(",")])
    is_pl = x['Model Class'] in ["PipelineModel"]
    class_to_call = x['Model Class'] if not is_pl else "PretrainedPipeline"
    meth_to_call = "pretrained" if not is_pl else "downloadPipeline"
    fun = f"{class_to_call}"
    if not is_pl:
        fun = fun+f".{meth_to_call}"
    language = f"Some({x.language})" if scala and is_pl else x.language
    language = x.language
    c = f'model = {fun}("{x["name"]}","{language}","{x.repo}")'
    if not is_pl:
        output_labels = x['Output Labels']
        c += f'\\\n\t.setInputCols({ins})\\\n\t.setOutputCol("{output_labels}")'
    else:
        c+='\n\nmodel.annotate("Include a healthcare document here. Can be a prescription, medical note, anything...")'
    return "val "+c.replace("\\","") if scala else c

def generate_buttons(x):
    ret = "{:.btn-box}\n"
    if x.demo_url:
        ret = ret + f"[Live Demo]({x.demo_url})"+"{:.button.button-orange}<br/>"
    else:
        ret = ret + "<button class=\"button button-orange\" disabled>Live Demo</button><br/>"
    if x.colab_url:
        ret = ret + f"[Open in Colab]({x.colab_url})"+"{:.button.button-orange.button-orange-trans.co.button-icon}<br/>"
    else:
        ret = ret + "<button class=\"button button-orange\" disabled>Open in Colab</button><br/>"
    if x.download_url:
        ret = ret + f"[Download]({x.download_url})"+"{:.button.button-orange.button-orange-trans.arr.button-icon}<br/>"
    else:
        ret = ret + "<button class=\"button button-orange\" disabled>Download</button><br/>"
    return ret
                                   
full_meta["python_sample"] = full_meta.apply(generate_code, axis=1)
#full_meta["python_sample"] =""
# full_meta["scala_sample"] = full_meta.apply(lambda x: generate_code(x, True), axis=1)
#full_meta["scala_sample"]=""
full_meta["buttons"] = full_meta.apply(generate_buttons, axis=1)
#full_meta["buttons"] = full_meta.apply(generate_buttons, axis=1)
# full_meta["real_labels"] = full_meta.apply(lambda r: None if r.model_class!="NerDLModel" else NerDLModel.pretrained(r["name"],r.language,r.repo).getClasses(), axis=1)

In [93]:

full_meta_tbl = full_meta.rename(columns={'name': 'Model Name', 
                                         'compatibility': 'Spark Compatibility',
                                         'spark_version': 'Spark NLP Compatibility',
                                         'license':'License',
                                         'edition':'Edition',
                                         'language': 'Language',
                                         'dimension': 'Dimension',
                                         'case_sensitive':'Case Sensitive',
                                         'upstream_deps':'Upstream Dependencies'})
def tabulate_row(x):
    if "dimension" in x.index:
        x["dimension"] = None if not x["dimension"] or pd.isna(x["dimension"]) else str(int(x["dimension"]))
    df = pd.DataFrame(x).dropna(how='any')
    df = df.replace(r'^\s*$', np.nan, regex=True)
    df = df.dropna(how='any')
    if  'Case Sensitive' in df:
        df['Case Sensitive'] = df['Case Sensitive'].astype(int)
        df[df['Case Sensitive'] >= 1] = 'True'
        df[df['Case Sensitive'] < 1] = 'False'
    return tabulate(df,tablefmt="github")



full_meta["table"] = \
    full_meta_tbl[["Model Name","Model Class","Spark Compatibility",
               "Spark NLP Compatibility","License","Edition","Input Labels",
               "Output Labels","Language","Dimension","Case Sensitive",
               "Upstream Dependencies"]]\
    .apply(tabulate_row, axis=1)

In [94]:
import datetime
for i, r in full_meta.iterrows():
    #print (r)
    with open(f"./temp_mds/{datetime.datetime.strptime(r.latest_date, '%d/%m/%Y').strftime( '%Y-%m-%d')}-{r['name']}_{r['language']}.md","w") as f:
        f.write(mdmd.render(**r))

In [ ]:
import pandas as pd
import numpy as np
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
spark = sparknlp.start()

In [ ]:
def run(text, model, lang):
    from googletrans import Translator
    translator = Translator()
    text = translator.translate(text, 
                     dest=lang).text

    documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

    sentenceDetector = SentenceDetector()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

    tokenizer = Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")

    if '840B_300' in model:
        embedding = 'glove_840B_300'
        lng = 'xx'
    elif '6B_300' in model:
        embedding = 'glove_6B_300'
        lng = 'xx'
    else:
        embedding = 'glove_100d'
        lng = 'en'
    embeddings = WordEmbeddingsModel.pretrained(embedding, lang=lng).\
                        setInputCols(["sentence", 'token']).\
                        setOutputCol("embeddings")

    public_ner = NerDLModel.pretrained(model, lang=lang) \
              .setInputCols(["sentence", "token", "embeddings"]) \
              .setOutputCol("ner")

    ner_converter = NerConverter() \
                    .setInputCols(["sentence", "token", "ner"]) \
                      .setOutputCol("ner_chunk")

    nlpPipeline = Pipeline(stages=[ documentAssembler, sentenceDetector,
                                     tokenizer,
                                     embeddings,
                                     public_ner,
                                     ner_converter
                                     ])

    empty_df = spark.createDataFrame([['']]).toDF("text")

    pipelineModel = nlpPipeline.fit(empty_df)
    lmodel = LightPipeline(pipelineModel)
    
    
    t_res = lmodel.fullAnnotate(text)[0]
    single_arr = []
    for r in t_res['ner_chunk']:
        single_arr.append({'ner_chunk':r.result,
                          'begin': r.begin,
                          'end':r.end,
                          'entity':r.metadata['entity']})
        
        
    code_sample = f"""
    embeddings = WordEmbeddingsModel.pretrained({embedding}, lang={lng})
    embeddings.setInputCols(["sentence", 'token'])
    embeddings.setOutputCol("embeddings")

    ner = NerDLModel.pretrained({model}, lang={lang})
    ner.setInputCols(["sentence", "token", "embeddings"])
    ner.setOutputCol("ner")

    ner_converter = NerConverter()
    ner_converter.setInputCols(["sentence", "token", "ner"])
    ner_converter.setOutputCol("ner_chunk")
                      
    pipeline = Pipeline(stages=[ documentAssembler, 
                                    sentenceDetector,
                                    tokenizer,
                                    embeddings,
                                    ner,
                                    ner_converter
                                     ])
    
    pipeline_model = pipeline.fit(spark.createDataFrame([['']]).toDF("text"))
    lmodel = LightPipeline(pipeline_model)
    
    result = lmodel.fullAnnotate("{text}")[0]
    """
    return pd.DataFrame(single_arr).to_markdown(tablefmt="grid", index=False), code_sample

#ex = """The Mona Lisa is a 16th century oil painting created by Leonardo. It's held at the Louvre in Paris."""
#lmodel = run(ex, 'dane_ner_840B_300','da')

In [ ]:
resses = []
code_resses = []
errrors = []
txt = u"""The Mona Lisa is a 16th century oil painting created by Leonardo. It's held at the Louvre in Paris."""
for i, row in full_meta_f.iterrows():
    try:
        re, co = run(txt, row['name'], row['language'])
        resses.append(re)
        code_resses.append(co)
    except:
        resses.append('')
        code_resses.append('')
        errrors.append(row['name']+'_'+row['language'])

In [ ]:
gc = bc = 0
for r_ in resses:
    if r_ =='':
        bc += 1
    else:
        gc += 1
print (bc, gc)

In [ ]:
full_meta['automated_output_generation'] = ''
full_meta['python_sample'] = ''
full_meta.loc[(full_meta['include']==1) & (full_meta['model_class']=='NerDLModel'), 'automated_output_generation'] = resses
full_meta.loc[(full_meta['include']==1) & (full_meta['model_class']=='NerDLModel'), 'python_sample'] = code_resses

full_meta.to_csv('docs_module/metadata/all_models_metadata.csv', index=False)

In [ ]:
errrors